In [1]:
import sys
sys.path.append("../../")
from STK_Sim import *

Filename = 'Communications_Setup'

stk_object = STK_Simulation(True,Filename)

In [2]:
# Comparing high data rate w/ no interpolation to low data rate w/ interpolation data

stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "UTCG")
start_time = time_convert(stk_object.root.CurrentScenario.StartTime)
duration = datetime.timedelta(days=1, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
stop_time=(start_time+duration).strftime("%d %b %Y %H:%M:%S.%f")
stk_object.root.CurrentScenario.StopTime=stop_time
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "EpSec")

# Loading targets into stk from file.
# targets = stk_object.Target_Loader("../../Input_Files/Targets_File.txt")
# sats = stk_object.Satellite_Loader("../../Input_Files/Satellites_File.txt")

## Facility

In [3]:
# IAgStkObjectRoot root: STK Object Model Root
facility = stk_object.root.CurrentScenario.Children.New(AgESTKObjectType.eFacility, 'USHI02') # eFacility

# IAgFacility facility: Facility Object
facility.Position.AssignGeodetic(19.013889, -155.663056,0.0) # Latitude, Longitude, Altitude

# Set altitude to height of terrain
facility.UseTerrain = True

# Set altitude to a distance above the ground
facility.HeightAboveGround = 0.92  # km

In [4]:
facility.SetAzElMask(1,0) # eTerrainData
azelMask = facility.Graphics.AzElMask
azelMask.RangeVisible = True
azelMask.NumberOfRangeSteps = 10
azelMask.DisplayRangeMinimum = 630   # km
azelMask.DisplayRangeMaximum = 4000  # km
azelMask.RangeColorVisible = True
#azelMask.RangeColor = 16776960 # cyan

#### Transmission

In [5]:
transmitter = facility.Children.New(AgESTKObjectType.eTransmitter, 'GSTransmitter')
transmitter.SetModel('Complex Transmitter Model')
txModel = transmitter.Model
txModel.SetModulator('BPSK')
txModel.Frequency = 7.19 # GHz
txModel.Power = 34.77 # dBW
txModel.DataRate = 56e-3 # Mb/sec

In [6]:
antennaControltx = txModel.AntennaControl
antennaControltx.SetEmbeddedModel('Parabolic')
antennaControltx.EmbeddedModel.DesignFrequency = 7.19 #GHz
antennaControltx.EmbeddedModel.InputType = 0
antennaControltx.EmbeddedModel.Beamwidth = 0.19 # deg
antennaControltx.EmbeddedModel.Efficiency = 100
antennaControltx.EmbeddedModel.UseBacklobeAsMainlobeAtten  = True

In [7]:
txModel.EnablePolarization = True
txModel.SetPolarizationType(2)
# txModel.Polarization
#dir(txModel)

In [8]:
accessConstraints = transmitter.AccessConstraints

In [9]:
constraintArray = accessConstraints.AvailableConstraints()
print('List of Available Constraints:')
for i in range(0,len(constraintArray)):
   print(constraintArray[i])

List of Available Constraints:
['3DTilesMask', 221]
['Altitude', 2]
['AngleOffBoresight', 75]
['AngularRate', 3]
['ApparentTime', 4]
['AtmosLoss', 148]
['AzElMask', 68]
['AzimuthAngle', 6]
['AzimuthRate', 69]
['BER+I', 149]
['BitErrorRate', 150]
['C/I', 151]
['C/N', 152]
['C/N+I', 153]
['C/No', 154]
['C/No+Io', 155]
['CbObstruction', 91]
['CentralAngle', 222]
['CentralDistance', 223]
['CloudsFogLoss', 156]
['CrdnAngle', 9]
['CrdnCondition', 104]
['CrdnVectorMag', 10]
['DeltaT/T', 158]
['DopplerShift', 159]
['Duration', 13]
['Eb/No', 160]
['Eb/No+Io', 161]
['ElevationAngle', 14]
['ElevationRate', 70]
['FluxDensity', 162]
['FreeSpaceLoss', 177]
['Frequency', 163]
['G/T', 164]
['GeoExclusion', 71]
['GMT', 16]
['GroundSampleDistance', 72]
['HeightAboveHorizon', 73]
['Intervals', 22]
['IonoFadingLoss', 0]
['J/S', 165]
['Lighting', 25]
['LineOfSight', 26]
['LinkEIRP', 166]
['LinkMargin', 167]
['LocalTime', 27]
['LOSLunarExclusion', 28]
['LOSSunExclusion', 29]
['LOSSunIlluminationAngle', 187]

In [10]:
freq = accessConstraints.AddNamedConstraint('Frequency')
freq.EnableMin = True
freq.EnableMax = True
freq.Min = 7.025 # GHz
freq.Max = 7.2 # GHz

In [11]:
linkEIRP = accessConstraints.AddNamedConstraint('LinkEIRP')
linkEIRP.EnableMax = True
linkEIRP.Max = 86

In [12]:
#gain = txModel.PostTransmitGainsLosses.Add(57.8) # dBi
#gain.Identifier = 'Transmitting Gain'

#### Receiving

In [13]:
receiver = facility.Children.New(AgESTKObjectType.eReceiver, 'GSReceiver')
receiver.SetModel('Complex Receiver Model')
recModel = receiver.Model


In [14]:
recModel.AutoTrackFrequency = False
recModel.Frequency = 8.35 # GHz

In [15]:
recModel.LinkMargin.Enable = True
recModel.LinkMargin.Type = 1
recModel.LinkMargin.Threshold = 10.5 # dB

In [16]:
antennaControlrx = recModel.AntennaControl
antennaControlrx.SetEmbeddedModel('Parabolic')

In [17]:
antennaControlrx.EmbeddedModel.UseBacklobeAsMainlobeAtten  = True

In [18]:
antennaControlrx.EmbeddedModel.DesignFrequency = 8.35 #GHz
antennaControlrx.EmbeddedModel.InputType = 0
antennaControlrx.EmbeddedModel.Beamwidth = 0.17 # deg
antennaControlrx.EmbeddedModel.Efficiency = 100

In [19]:
gain = recModel.PreReceiveGainsLosses.Add(59.1) # dBi
gain.Identifier = 'Receiving Gain'

In [20]:
recModel.EnablePolarization = True
recModel.SetPolarizationType(2)
# recModel.Polarization

In [21]:
accessConstraints2 = receiver.AccessConstraints

In [22]:
systemSensitivity = accessConstraints2.AddNamedConstraint('G/T')
systemSensitivity.EnableMax = True
systemSensitivity.Max = 37.7 # db/K

In [23]:
freq2 = accessConstraints2.AddNamedConstraint('Frequency')
freq2.EnableMin = True
freq2.EnableMax = True
freq2.Min = 8.0 # GHz
freq2.Max = 8.5 # GHz

In [24]:
# recModel.SystemNoiseTemperature.ConstantNoiseTemperature = 220 # K  

In [25]:
facilities = [facility]

## Coverage

In [26]:
dir(stk_object)

['Compute_AzEl',
 'Generate_Pre_Planning_Data',
 'Get_Access_DP',
 'Get_Satellite_DP',
 'Interpolate',
 'Plan',
 'Reset_Target_Bins',
 'Satellite_Loader',
 'Target_Loader',
 'Update_Mass_Properties',
 'Update_Target_Bins',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'dt',
 'root',
 'scenario',
 'set_sim_time',
 'stk']

In [27]:
stk_object.Satellite_Loader("../../Input_Files/Satellites_File.txt")
stk_object.dt = 3600
stk_object.set_sim_time(weeks=4)
uplink = stk_object.Get_Access_DP(facilities, stk_object.satellites, "AER Data/Default", ['Time','Elevation'])

- Computing_AER Data/Default [==========] 10/10 [100%] in 1:00.1 (0.16/s)       


In [28]:
uplink = stk_object.Get_Access_DP(facilities, stk_object.satellites, "AER Data/Default", ['Time','Elevation'])

- Computing_AER Data/Default [          ] ▁▃▅ 0/10 [0%] in 0s (~0s, 0.0/s) 

- Computing_AER Data/Default [==========] 10/10 [100%] in 1:04.9 (0.15/s)       


In [29]:
uplink[0][1]

{'Time': [2249.395157163,
  2564.851193023,
  2880.307228882,
  40540.869794182,
  40920.023953697,
  41299.178113213,
  46461.785876962,
  46707.003326095,
  46952.220775228,
  83439.793594425,
  83815.763205592,
  84191.732816759,
  89302.583826625,
  89573.242751305,
  89843.901675985,
  122101.79726081,
  122192.236533988,
  122282.675807166,
  127548.584259945,
  127933.369573406,
  128318.154886867,
  133575.767299401,
  133727.480243226,
  133879.193187051,
  170447.227875575,
  170831.096608272,
  171214.965340969,
  176377.536873637,
  176580.089233466,
  176782.641593294,
  208983.66857752,
  209197.617006938,
  209411.565436356,
  214564.567376454,
  214947.799654328,
  215331.031932202,
  251883.39964476,
  252052.243935395,
  252221.08822603,
  257460.941921514,
  257845.380910974,
  258229.819900435,
  263534.545344282,
  263585.227820463,
  263635.910296645,
  295926.876964145,
  296204.682017413,
  296482.487070681,
  301588.978616431,
  301963.274737742,
  302337.57085

In [30]:
downlink = stk_object.Get_Access_DP(stk_object.satellites, facilities, "Targeting Data", ['Time','SlantRange'])

- Computing_Targeting Data [          ] ▁▃▅ 0/10 [0%] in 0s (~0s, 0.0/s) 

- Computing_Targeting Data [==========] 10/10 [100%] in 57.7s (0.16/s)          


In [31]:
downlinkelevation = stk_object.Get_Access_DP(stk_object.satellites, facilities, "AER Data/Default", ['Time','Elevation'])

- Computing_AER Data/Default [==========] 10/10 [100%] in 1:08.2 (0.14/s)       


In [32]:
downlink[0][0] # Sat_0 example 

{'Time': [34764.367854464,
  35127.09691677,
  35489.825979077,
  40597.344999061,
  40905.43535713,
  41213.525715199,
  77661.622610664,
  78017.68830436,
  78373.753998055,
  83462.274312832,
  83784.456564314,
  84106.638815797,
  121761.759224871,
  122139.068530036,
  122516.377835202,
  127667.354372987,
  127924.597943147,
  128181.841513307,
  164660.608753326,
  165034.220271953,
  165407.831790581,
  170512.659732037,
  170793.098518508,
  171073.537304979,
  203389.020054796,
  203412.937913966,
  203436.855773136,
  208767.909378493,
  209152.189325355,
  209536.469272216,
  214770.431748244,
  214944.840009311,
  215119.248270377,
  251666.785722197,
  252049.730784472,
  252432.675846748,
  257582.136691505,
  257800.230084527,
  258018.32347755,
  290219.650200109,
  290417.968805375,
  290616.287410641,
  295782.350434546,
  296166.401260853,
  296550.452087161,
  333125.351830822,
  333269.854745378,
  333414.357659934,
  338679.325573893,
  339064.194069366,
  339449

In [33]:
downlinkelevation[0][0]

{'Time': [34764.367854464,
  35127.09691677,
  35489.825979077,
  40597.344999061,
  40905.43535713,
  41213.525715199,
  77661.622610664,
  78017.68830436,
  78373.753998055,
  83462.274312832,
  83784.456564314,
  84106.638815797,
  121761.759224871,
  122139.068530036,
  122516.377835202,
  127667.354372987,
  127924.597943147,
  128181.841513307,
  164660.608753326,
  165034.220271953,
  165407.831790581,
  170512.659732037,
  170793.098518508,
  171073.537304979,
  203389.020054796,
  203412.937913966,
  203436.855773136,
  208767.909378493,
  209152.189325355,
  209536.469272216,
  214770.431748244,
  214944.840009311,
  215119.248270377,
  251666.785722197,
  252049.730784472,
  252432.675846748,
  257582.136691505,
  257800.230084527,
  258018.32347755,
  290219.650200109,
  290417.968805375,
  290616.287410641,
  295782.350434546,
  296166.401260853,
  296550.452087161,
  333125.351830822,
  333269.854745378,
  333414.357659934,
  338679.325573893,
  339064.194069366,
  339449

In [34]:
LLA_State = stk_object.Get_Satellite_DP("LLA State/Fixed")

- Computing_LLA State/Fixed [          ] ▁▃▅ 0/10 [0%] in 0s (~0s, 0.0/s) 

- Computing_LLA State/Fixed [==========] 10/10 [100%] in 12.0s (0.80/s)         


In [35]:
LLA_State

[          time        lat           lon         alt     alt agl  lat rate  \
 0          0.0   0.034493 -1.911716e-14  600.000008  599.982844  0.062426   
 1       3600.0 -43.599866  1.638866e+02  610.128336  610.128495 -0.062052   
 2       7200.0  86.659704 -4.977122e+01  621.311519  621.291202  0.058062   
 3      10800.0 -49.923018 -4.378330e+01  612.480202  612.475770  0.061962   
 4      14400.0   6.399973  1.199628e+02  600.263666  600.207308 -0.062416   
 ..         ...        ...           ...         ...         ...       ...   
 692  2491200.0  26.725959 -1.478503e+02  604.298190  604.313303 -0.062268   
 693  2494800.0  16.904345  1.619858e+01  601.795182  601.560252  0.062360   
 694  2498400.0 -60.342538  1.795201e+02  616.117927  616.164756 -0.061814   
 695  2502000.0  76.422164  1.711312e+02  620.197347  620.194110 -0.061502   
 696  2505600.0 -33.140698 -2.784613e+01  606.356453  606.349779  0.062192   
 
      lon rate  alt rate  centric lat   centric lon  terrain a